# **Feature-wise Model Segmentation: Predicting Credit Card Usage**


## **Summary**

The objective of this project is to produce a model that can accurately predict whether or not a particular user, on a given day, is going to have a credit card tranaction in the next k-days. This project loads the raw dataset and, upon data cleaning, retains 89% of the original rows. Additionally, users must have made at least 2 transactions over 100 days, which further sheds another 3.2%. 

The model developed is infact a collection of 3578 XGBoost Classifiers assembled together. This ensemble approach works by training each individual model on a specific user's transaction data and developing predictions for that user. For k=5 and k=10, this approach significantly outperformed the naive baseline, Logistic Regression, and a single XGBoost Classifier trained on the entire dataset. 

## **Methodology**
* **Data Loading**: Load data from csv and clean the raw data
* **Data Preparation**: 
    * DataFrame: Produce a dataframe containing the daily history of each person and whether or not they had a transaction that day  
    * Target: Create a function that adds a target column, identifying for whether a given user on a particular day makes at least one transaction in the next k days (exclusive of the current day)
    * Feature Extraction: Extract additional features such as cumulative transactions, rolling transactions, Day of Week, etc., and apply One-hot-encoding to categorical variables   
* **ML Pipeline**: 
    Evaluate the following approaches
    *  Standard Approach: train a model (Logistic or XGBoost) on the dataset, obtain test predictions
    *  Ensemble Approach: Partition the dataset by each unique Person, train individual models on each subset, and collect all test predictions
        
* **Evaluation**: Identify the best performing approach based on several evaluation metrics.

## **Model Choice**

XGBoost was chosen as the primary model for this project. Logistic regression was also implemented to serve as a reference and comparison.

This is due to the following reasons:

* Adaptive learning approach: Unlike random forests, which build trees independently, XGBoost constructs trees iteratively, with each new tree focusing on correcting the errors of the previous ensemble. This allows XGBoost to capture more complex patterns and interactions in the data.

* Regularization: These techniques were extremely important given the numerous categorical variables that required one-hot-encoding. Regularization prevents overfitting, which can be a concern with random forests. 

* Gradient boosting framework: This enables using a log-loss objective function, leading to better performance

## **Results**

Below are the evaluation metrics on the test dataset using the final Ens.XGB model. 

| Name          | Accuracy | PCC Baseline | F1  | Precision | Recall | AUROC | AvgPrecision |
|---------------|----------|--------------|-----|-----------|--------|-------|--------------|
| Ens.XGB k=1   | 0.81     | 0.71         | 0.38| 0.47      | 0.31   | 0.74  | 0.41         |
| Ens.XGB k=5   | 0.89     | 0.50         | 0.90| 0.89      | 0.90   | 0.95  | 0.94         |
| Ens.XGB k=10  | 0.95     | 0.59         | 0.96| 0.96      | 0.96   | 0.98  | 0.99         |


# **Execution**

## **Libraries**

In [1]:

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Machine learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, auc,
    precision_score, recall_score,
    average_precision_score, roc_curve,
    silhouette_score
)
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

# Misc
from tqdm import tqdm
import zipfile
import io
import warnings

warnings.filterwarnings("ignore")

## **Data Loading**

The data was loaded from `cc_transactions.csv`. Data was checked for nulls, duplicates, negative amounts, and incorrect dates. Negative amounts were excluded as these are likely not transactions but refunds. Incorrect dates were determined on the basis of the posting date exceeding the transaction dates. 

After this cleaning, 89.5% of data was retained.

In [2]:
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    with zip_ref.open('cc_transactions.csv') as file:
        csv_file = io.TextIOWrapper(file)
        df = pd.read_csv(csv_file, parse_dates=['Transaction Date', 'Posted Date'])

df.head(3)

,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Description,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC)
0,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,GENERAL PURCHASE,890.00,NACAS,2013-07-30,2013-07-31,CHARITABLE AND SOCIAL SERVICE ORGANIZATIONS
1,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,ROOM CHARGES,368.96,SHERATON HOTEL,2013-07-30,2013-07-31,SHERATON
2,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,J,GENERAL PURCHASE,165.82,SEARS.COM 9300,2013-07-29,2013-07-31,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...


In [3]:
original_rows = df.shape[0]
current_rows = df.shape[0]

# Drop rows with nulls in important columns 
df = df.dropna(subset=['Amount', 'Transaction Date', 'Posted Date', 'Cardholder Last Name'])
null_rows = current_rows - df.shape[0]
current_rows =  df.shape[0]

# Drop rows with nulls in important columns 
df = df.drop_duplicates()
duplicate_rows = current_rows - df.shape[0]
current_rows =  df.shape[0]

# Drop rows with negatives
df = df[df['Amount'] > 0]
negative_rows = current_rows - df.shape[0]
current_rows =  df.shape[0]

# Drop rows where Posted Date < Transaction Date
df = df[df['Posted Date'] > df['Transaction Date']]
incorrectdate_rows = current_rows - df.shape[0]
current_rows =  df.shape[0]

print('Original Rows: ', original_rows)
print('Null Rows Dropped: ', null_rows)
print('Duplicate Rows Dropped: ', duplicate_rows)
print('Negative Rows Dropped: ', negative_rows)
print('Incorrect Date Rows Dropped: ', incorrectdate_rows)
print('----')
print('Final Rows: ', current_rows)
print(f'Retention: {current_rows/original_rows*100:.1f}%')

Original Rows:  442458
Null Rows Dropped:  0
Duplicate Rows Dropped:  15048
Negative Rows Dropped:  13814
Incorrect Date Rows Dropped:  17767
----
Final Rows:  395829
Retention: 89.5%


## **Data Prepreparation**

### Reduce

Columns which will not be used for model training were removed. This includes categorical variables of high cardinality, which would present problems in model training and overfitting. Amount was also not considered in this project as prior EDA suggests no relationship with card credit usage frequency.

In [4]:
df['Agency Number'] = df['Agency Number'].astype(str)
df['PersonID'] = df['Agency Number'] \
    + '-' + df['Cardholder Last Name'] \
    + '-' + df['Cardholder First Initial']
columns = ['PersonID', 'Transaction Date']
df = df[columns]
df.columns = ['PersonID', 'Date']
df.head(3)

,PersonID,Date
0,1000-Mason-C,2013-07-30
1,1000-Mason-C,2013-07-30
2,1000-Massey-J,2013-07-29


### Explode

The code below transforms the dataframe into an exploded form. Each row represents a unique Person-Date combination, with the date range determined by the earliest and latest transaction for that particular individual. 

In [5]:
columns = ['PersonID', 'Date']
df = df[columns].drop_duplicates(subset=columns)
df['Date'] = pd.to_datetime(df['Date'])

df_temp0 = df.groupby('PersonID').agg({
    'Date': ['min', 'max']
})

date_range = lambda row: pd.date_range(start=row['Date']['min'],
                                       end=row['Date']['max'], freq='D')
df_temp1 = df_temp0.apply(date_range, axis=1).explode().reset_index()
df_temp1.columns = ['PersonID', 'Date']
df_temp1 = df_temp1.reset_index(drop=True)

df_temp1 = df_temp1.merge(df[['PersonID', 'Date']], 
                          on=['PersonID', 'Date'], 
                          how='left', 
                          indicator='has_transaction')
df_temp1['has_transaction'] = (df_temp1['has_transaction'] == 'both').astype(int)

df_temp1.head(10)

,PersonID,Date,has_transaction
0,1000-AVERY-K,2013-10-22,1
1,1000-AVERY-K,2013-10-23,0
2,1000-AVERY-K,2013-10-24,0
3,1000-AVERY-K,2013-10-25,0
4,1000-AVERY-K,2013-10-26,0
5,1000-AVERY-K,2013-10-27,0
6,1000-AVERY-K,2013-10-28,0
7,1000-AVERY-K,2013-10-29,0
8,1000-AVERY-K,2013-10-30,0
9,1000-AVERY-K,2013-10-31,0


### Extract Features

In [6]:
def past_n_days_purchases(group, n):
    """Calculates the number of days with transaction(s) in the last n-days"""
    return group.sort_values('Date').rolling(window=f'{n}D', on='Date')['has_transaction'] \
        .sum().fillna(0).astype(int)

In [7]:
df_temp2 = df_temp1.copy()

# Get date related features
min_date = df_temp1['Date'].min()
df_temp2['DayIndex'] = (df_temp2['Date'] - min_date).dt.days
df_temp2['runningTotal'] = df_temp2.sort_values('Date').groupby('PersonID')['has_transaction'].cumsum()
df_temp2['Month'] = df_temp2['Date'].dt.strftime('%b')
df_temp2['DayOfWeek'] = df_temp2['Date'].dt.strftime('%a')
df_temp2['Year'] = df_temp2['Date'].dt.strftime('%Y')

# Get rolling purchases 
df_temp2['purchases_PastWeek'] = df_temp2.groupby('PersonID') \
    .apply(lambda x: past_n_days_purchases(x, 7), include_groups=False) \
        .reset_index(drop=True)

# Filter
df_temp2 = df_temp2.loc[df_temp2.runningTotal >= 7]

# Preview
df_temp2.head(3)

,PersonID,Date,has_transaction,DayIndex,runningTotal,Month,DayOfWeek,Year,purchases_PastWeek
82,1000-Abbott-T,2013-08-02,1,78,7,Aug,Fri,2013,3
83,1000-Abbott-T,2013-08-03,0,79,7,Aug,Sat,2013,3
84,1000-Abbott-T,2013-08-04,0,80,7,Aug,Sun,2013,3


In [8]:
dummy_cols = ['Month','DayOfWeek']
df_temp3 = pd.get_dummies(df_temp2, columns=dummy_cols, drop_first=True, dtype=int)
df_temp3.reset_index(drop=True, inplace=True)
df_temp3.head()

,PersonID,Date,has_transaction,DayIndex,runningTotal,Year,purchases_PastWeek,Month_Aug,Month_Dec,Month_Feb,...,Month_May,Month_Nov,Month_Oct,Month_Sep,DayOfWeek_Mon,DayOfWeek_Sat,DayOfWeek_Sun,DayOfWeek_Thu,DayOfWeek_Tue,DayOfWeek_Wed
0,1000-Abbott-T,2013-08-02,1,78,7,2013,3,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000-Abbott-T,2013-08-03,0,79,7,2013,3,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1000-Abbott-T,2013-08-04,0,80,7,2013,3,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1000-Abbott-T,2013-08-05,1,81,8,2013,3,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1000-Abbott-T,2013-08-06,0,82,8,2013,3,1,0,0,...,0,0,0,0,0,0,0,0,1,0


### Get Target 
Ex. k=5

In [9]:
def get_target(df, k):
    """
    Function to assign a 'Target' value indicating the presence of transactions within a specified window (k days).

    Parameters:
    - df (pandas.DataFrame): DataFrame with columns 'PersonID', 'DayIndex', and 'has_transaction'.
    - k (int): Number of days to consider for the target window.

    Returns:
    - pandas.DataFrame: DataFrame with an additional 'Target' column.
    """
    
    # Sort values by 'PersonID' and 'DayIndex'
    df = df.sort_values(by=['PersonID', 'DayIndex'])
    
    # Initialize 'Target' column
    df['Target'] = 0

    # List to hold modified groups
    modified_groups = []
    
    # Process each group
    grouped = df.groupby('PersonID')
    for name, group in tqdm(grouped):
        # Identify the days where a transaction occurred
        transaction_days = group[group['has_transaction'] == 1]['DayIndex']

        # Use broadcasting to find all days within the k-day window of each transaction
        for day in transaction_days:
            # Define the mask within the group's 'DayIndex'
            mask = (group['DayIndex'] < day) & (group['DayIndex'] >= day - k)
            group.loc[mask, 'Target'] = 1  # Apply mask to the group

        # Append modified group to the list
        modified_groups.append(group)

    # Concatenate all modified groups back into a single DataFrame
    new_df = pd.concat(modified_groups, ignore_index=True)
    return new_df


In [10]:
k = 5
df = get_target(df_temp3, k)
cols = ['PersonID', 'Date', 'has_transaction', 'DayIndex', 'Target']
display(df[cols].tail(25))

100%|██████████| 4377/4377 [01:23<00:00, 52.32it/s] 


,PersonID,Date,has_transaction,DayIndex,Target
998290,98000-Young-J,2014-05-27,0,376,1
998291,98000-Young-J,2014-05-28,0,377,1
998292,98000-Young-J,2014-05-29,1,378,1
998293,98000-Young-J,2014-05-30,0,379,1
998294,98000-Young-J,2014-05-31,1,380,0
998295,98000-Young-J,2014-06-01,0,381,1
998296,98000-Young-J,2014-06-02,0,382,1
998297,98000-Young-J,2014-06-03,0,383,1
998298,98000-Young-J,2014-06-04,0,384,1
998299,98000-Young-J,2014-06-05,0,385,1


In [11]:
df = df.drop('Date', axis=1)
del df_temp0, df_temp1, df_temp2, df_temp3

## **Model Selection**

### **Functions**

In [12]:
def ml_pipeline(data, model):
    """Get predictions given data and model."""
    X, y = data.drop('Target', axis=1), data['Target']

    scaler = MinMaxScaler()
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=0.2,
                                                        random_state=42)
    X_train = scaler.fit_transform(X_train).astype(np.float32)
    X_test = scaler.transform(X_test).astype(np.float32)
    
    try:
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]
        
    # triggered in situations where all y_train == 1
    except: 
        y_proba = np.array([1]*len(X_test))
    return y_test, y_proba


def eval_metrics(y_test, y_proba, name=''):
    """Get evaluation metrics."""
    y_pred = np.where(y_proba >= 0.5, 1, 0)
    fpr, tpr, thr = roc_curve(y_test, y_proba)
    avgPrecision = average_precision_score(y_test, y_proba)
    roc = auc(fpr, tpr)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    pcc = sum((y_test.value_counts() / len(y_test))**2)
    
    result = {'Name': name,
                'Accuracy': accuracy,
                'PCC Baseline': pcc,
                'F1': f1,
                'Precision': precision,
                'Recall': recall,
                'AUROC': roc,
                'AvgPrecision': avgPrecision}
    
    return result

### Standard Approach

The standard approach is to train a model on the full training dataset. In this approach, the PersonID was not considered as a feature. 

In [13]:
df.iloc[:, 1:-1].columns

Index(['has_transaction', 'DayIndex', 'runningTotal', 'Year',
       'purchases_PastWeek', 'Month_Aug', 'Month_Dec', 'Month_Feb',
       'Month_Jan', 'Month_Jul', 'Month_Jun', 'Month_Mar', 'Month_May',
       'Month_Nov', 'Month_Oct', 'Month_Sep', 'DayOfWeek_Mon', 'DayOfWeek_Sat',
       'DayOfWeek_Sun', 'DayOfWeek_Thu', 'DayOfWeek_Tue', 'DayOfWeek_Wed'],
      dtype='object')

In [14]:
# k=5
 
results = []
data = df.iloc[:, 1:]

xgb_model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss',
                              random_state=42)
logit = LogisticRegression(max_iter=1000, random_state=42)
models = [(xgb_model, 'XGB'), (logit, 'Logit')]

for model, name in models:
    y_test, y_proba = ml_pipeline(data, model)
    result = eval_metrics(y_test, y_proba, name=name)
    results.append(result)

In [15]:
pd.DataFrame(results)

,Name,Accuracy,PCC Baseline,F1,Precision,Recall,AUROC,AvgPrecision
0,XGB,0.701938,0.501866,0.713962,0.727258,0.701142,0.776229,0.806156
1,Logit,0.689632,0.501866,0.697758,0.721799,0.675267,0.757134,0.783041


### Ensemble Approach

The ensemble approach is a collection of numerous individual ML models to form a collective system. Each ML model is trained specifically for a unique Person by using their unique subset of the data.

In [16]:
# Preprocessing, remove users with few rows
persons = df.PersonID.value_counts()
persons = [p for p, cnt in persons.items() if cnt > 99]
data_size = len(df.loc[df.PersonID.isin(persons)])

print(f"Filtering out users with less than 100 rows...")
print(f"Considering {data_size} out of {len(df)} rows ({data_size/len(df)*100:.1f}%)")

cols = df.columns[1:]
probas = {'XGB': [], 'Logit': []}

# Initialize ML pipeline for a given model
for model, name in models:
    actuals = []
    
    # Run ML Pipeline on each subset
    for person in tqdm(persons):
        # Get subset
        data = df.loc[df.PersonID == person, cols]
        
        # Get local predictions
        y_test, y_proba = ml_pipeline(data, model)
        
        # Store local predictions
        probas[name].extend(y_proba)
        actuals.extend(y_test)
    
    # Evaluate collective predictions
    result = eval_metrics(pd.Series(actuals), np.array(probas[name]), name='Ensemble-' + name)
    results.append(result)

# Get results
df_results = pd.DataFrame(results)

Filtering out users with less than 100 rows...
Considering 966494 out of 998315 rows (96.8%)


100%|██████████| 3578/3578 [07:40<00:00,  7.77it/s]


In [17]:
print("Evaluation metrics (k=5):")
df_results.round(2)

Evaluation metrics (k=5):


,Name,Accuracy,PCC Baseline,F1,Precision,Recall,AUROC,AvgPrecision
0,XGB,0.70,0.5,0.71,0.73,0.70,0.78,0.81
1,Logit,0.69,0.5,0.70,0.72,0.68,0.76,0.78
2,Ensemble-XGB,0.89,0.5,0.90,0.89,0.90,0.95,0.95
3,Ensemble-Logit,0.74,0.5,0.76,0.76,0.76,0.83,0.85


# **Evaluation**

Using the ensemble approach with XGBoost Classifier (Ens.XGB), different settings of k were evaluated. In k=5 and k=10, the Ens.XGB was able to significantly outperform the naive baseline in accuracy as calculated using the Proportional Chance Criterion. Other evaluations such as the Precision, Recall, and F1-score which do not weigh TN predictions, also demonstrate that this approach produces high quality positive predictions.

In [18]:
# Store previous result
results_k = df_results[df_results.Name=='Ensemble-XGB'] \
    .to_dict(orient='records')
results_k[0]['Name'] = 'Ens.XGB k=5'

In [19]:
model = xgb_model
ks = [(1, 'Ens.XGB k=1'), (10, 'Ens.XGB k=10')]
probas = {'Ens.XGB k=1': [], 'Ens.XGB k=10': []}

for k, name in ks:
    # Get target given K
    df = get_target(df, k)
    actuals = []
    
    # Train an ML model for each person, collectively forming an ensemble system
    for person in tqdm(persons):
        # Get subset 
        data = df.loc[df.PersonID == person, cols]
        
        # Get local predictions
        y_test, y_proba = ml_pipeline(data, model)
        
        # Store local predictions
        probas[name].extend(y_proba)
        actuals.extend(y_test)
        
    # Evaluate collective predictions
    result = eval_metrics(pd.Series(actuals), np.array(probas[name]), name=name)
    results_k.append(result)

# Get Results
df_results_k = pd.DataFrame(results_k).sort_values(by='Accuracy')
display(df_results_k.round(2))

100%|██████████| 3578/3578 [11:55<00:00,  4.86it/s]


,Name,Accuracy,PCC Baseline,F1,Precision,Recall,AUROC,AvgPrecision
1,Ens.XGB k=1,0.814826,0.706990,0.376468,0.471013,0.313533,0.738163,0.410986
0,Ens.XGB k=5,0.889685,0.501079,0.895150,0.890353,0.899999,0.946286,0.941916
2,Ens.XGB k=10,0.945210,0.585789,0.961377,0.958431,0.964342,0.976915,0.988114
